In [1]:
#NN Surrogate model class
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import gpytorch
import botorch 

import matplotlib.pyplot as plt

# BO for Minimizing Emittance*Bmag with 9 Variables (SQ, CQ, SOL, matching quads)

In [2]:
Model = Surrogate_NN(pytorch=True)

Model.load_saved_model(model_path = '../models/', 
                       model_name = 'Surrogate_NN_PyTorch')

Model.load_scaling(scalerfilex = '../data/transformer_x_pytorch.pth', 
                   scalerfiley = '../data/transformer_y_pytorch.pth')
Model.take_log_out = False

## Import design Twiss parameters (OTR2)

In [3]:
beamline_info = json.load(open('../configs/beamline_info.json'))
get_twiss0 = beamline_info['Twiss0']

# emit, beta, alpha
twiss0 = {'x': [get_twiss0[0], get_twiss0[2], get_twiss0[4]],
          'y': [get_twiss0[1], get_twiss0[3], get_twiss0[5]]}

beta0_x, alpha0_x = twiss0['x'][1], twiss0['x'][2]
beta0_y, alpha0_y = twiss0['y'][1], twiss0['y'][2]
# print(twiss0['x'], twiss0['y'])

## Objective Function

In [4]:
# convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

# input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','CQ01:b1_gradient', 'SQ01:b1_gradient',
                 "QA01:b1_gradient", "QA02:b1_gradient", 
                 "QE01:b1_gradient", "QE02:b1_gradient", "QE03:b1_gradient", "QE04:b1_gradient"]

bounds = torch.as_tensor([[0.46, 0.485], [-0.02, 0.02], [-0.02, 0.02],
                       [-4, -1], [1, 4],
                       [-7,-1], [-1, 7],[-1, 7], [-7, 1]])

# output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

In [5]:
def evaluate(config): 
    """
    D is input space dimensionality
    N is number of sample points
    :param config: input values of opt_var_names, torch.tensor, shape (N, D) 
    returns (N, 1) 
    """
    N = config.shape[0]
    D = config.shape[1]
    
    # make input array of length model_in_list (inputs model takes)
    x_in = torch.empty((N,len(Model.model_in_list)))
    
    # fill in reference point around which to optimize
    x_in[:,:] = torch.tensor(ref_point[0])

    #set solenoid, CQ, SQ, matching quads to values from optimization step
    col = []
    for i in range(D):
        col.append(Model.loc_in[opt_var_names[i]]) #should make col a flat list of indices, e.g. [4, 6, 7]
    x_in[:, col] = config[:,:] 
    
    #output predictions
    y_out = Model.pred_machine_units(x_in)

    return -1*objective(y_out)


def objective(y_out):
    """
    :param y_out: tensor with has a shape of (N, num_outputs)
    returns tensor of emittance * bmag for each input, shape (N, 1)
    """
    
    # geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    emit = torch.sqrt(out1 * out2)
  
    sigma_x = y_out[:,Model.loc_out['sigma_x']] #grab sigma_x out of the model 
    sigma_y = y_out[:,Model.loc_out['sigma_y']] #grab sigma_y out of the model 
    
    # real beta and alpha 
    # NEEDS TO BE FIXED - currently assuming real alpha to be the same as design alpha 
    alpha_x = torch.tensor(alpha0_x).repeat(y_out.shape[0])
    alpha_y = torch.tensor(alpha0_y).repeat(y_out.shape[0])
    beta_x, beta_y = (sigma_x**2) / out1, (sigma_y**2) / out2
    
    # bmag 
    bmag_x = 0.5 * ((beta0_x / beta_x) + (beta_x / beta0_x)) + 0.5 * ((alpha_x * torch.sqrt(beta0_x / beta_x) - alpha0_x * torch.sqrt(beta_x / beta0_x))**2)
    bmag_y = 0.5 * ((beta0_y / beta_y) + (beta_y / beta0_y)) + 0.5 * ((alpha_y * torch.sqrt(beta0_y / beta_y) - alpha0_y * torch.sqrt(beta_y / beta0_y))**2)
    bmag = torch.sqrt(bmag_x * bmag_y)
    
    out = (emit * bmag)/1e-6 # in um units 
    return out.reshape(-1,1)

## Example: Set up initial training samples

In [6]:
#create initial samples within specified bounds
n_samples = 3
n_var = 9

train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.tensor(np.random.uniform(bounds[i,0],bounds[i,1],(n_samples,)))
print(train_x)

train_y = evaluate(train_x).detach() # detach gradient to fix going backwards twice error while fitting hyperparameters
print(train_y)

tensor([[ 4.8271e-01,  8.0343e-03,  1.2452e-02, -2.6051e+00,  3.1735e+00,
         -5.4950e+00,  3.4614e+00,  4.7547e+00, -2.9873e+00],
        [ 4.7396e-01, -6.6409e-04, -1.9101e-02, -2.6881e+00,  2.7658e+00,
         -4.5992e+00,  3.6788e-01,  1.2695e+00, -4.5670e+00],
        [ 4.8126e-01,  1.4494e-02,  6.8171e-03, -3.3674e+00,  1.4442e+00,
         -4.7933e+00, -8.6104e-01,  4.0675e+00, -6.6396e+00]])
tensor([[ -2.1410],
        [-11.9202],
        [ -3.9883]])


## Bayesian Optimization

### Gaussian Regression & Acquisition Function

In [7]:
def get_BO_point(x, f, bounds, beta=2.0, mean_module=None, input_transform=None, outcome_transform=None):
    """
    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    gp = botorch.models.SingleTaskGP(x, f,
                                     mean_module=mean_module, 
                                     outcome_transform=outcome_transform, 
                                     input_transform=input_transform)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=100,
                                                       raw_samples=100)
    return candidate, gp

In [8]:
def BayesianOptimization(train_x, train_y, n_steps, prior=None, transformer_x=None, transformer_y=None):
    best_y_list = []
    best_y = torch.max(train_y)
    best_y_list.append(best_y)
    
    for i in range(n_steps):
        x_new, model = get_BO_point(train_x, train_y, 
                                    bounds=bounds.transpose(0,1), 
                                    mean_module=prior, 
                                    input_transform=transformer_x, 
                                    outcome_transform=transformer_y)

        train_x = torch.cat((train_x, x_new))
        new_y = evaluate(train_x[-1].reshape(1,-1)).detach()
        train_y = torch.cat((train_y, new_y))

        if (new_y > best_y):
            best_y = new_y
            color = '\033[95m', '\033[0m'
        else: 
            color = '\u001b[30m', '\033[0m'
        
        best_y_list.append(best_y)
        
#         print("iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4")
#         print(f'{color[0]}{i+1}      {train_y[-1,0]:.5f}   {train_x[-1,0]:.5f}   {train_x[-1,1]:.5f}   {train_x[-1,2]:.5f}   {train_x[-1,3]:.5f}   {train_x[-1,4]:.5f}   {train_x[-1,5]:.5f}   {train_x[-1,6]:.5f}   {train_x[-1,7]:.5f}   {train_x[-1,8]:.5f}{color[1]}')
        
    return torch.tensor(best_y_list), train_x

### Custom Mean Modules

In [9]:
# class used to create NN custom prior
import torch.nn as nn
class NN5_prior(nn.Module):
    def __init__(self):
        super(NN5_prior, self).__init__()
        
        hidden_size = 40
        self.network = nn.Sequential(nn.Linear(n_var, hidden_size), 
                                     nn.Tanh(), 
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, hidden_size),
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, hidden_size),
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, 1))
        
        
    def forward(self, x):
        x = self.network(x)
        return x 

class NN4_prior(nn.Module):
    def __init__(self):
        super(NN4_prior, self).__init__()
        
        hidden_size = 30
        self.network = nn.Sequential(nn.Linear(n_var, hidden_size), 
                                     nn.Tanh(), 
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, 1))
        
    def forward(self, x):
        """
        takes in transformed x, returns transformed y
        """
        x = self.network(x)
        return x 

class NN3_prior(nn.Module):
    def __init__(self):
        super(NN3_prior, self).__init__()
        
        hidden_size = 20
        self.network = nn.Sequential(nn.Linear(n_var, hidden_size), 
                                     nn.Tanh(), 
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, 1))
        
    def forward(self, x):
        """
        takes in transformed x, returns transformed y
        """
        x = self.network(x)
        return x 
    
# NN model trained from surrogate model samples as prior
from gpytorch.means.mean import Mean
class CustomMean(Mean):
    def __init__(self, name, prior, NN_y_transform, outcome_transform):
        super(CustomMean, self).__init__()
        self.NN_model = prior
        self.NN_model.load_state_dict(torch.load('./results/' + name + '.pth'))
        self.NN_model.eval()
        
        self.NN_model.requires_grad = False
        
        self.y_norm_transform = botorch.models.transforms.input.Normalize(1)
        self.y_norm_transform.load_state_dict(torch.load(f'./results/' + NN_y_transform + '.pth'))
        self.y_norm_transform.eval()
        
        self.outcome_transform = outcome_transform
        
    def forward(self, x):
        """
        takes in transformed x, returns transformed y
        """
        self.outcome_transform.eval()
        
        out = []
        if (x.dim() == 2):
            x = x.unsqueeze(dim=0) # shape x into (batch_size = 1, n_samples, n_var) if necessary 
        for i in range(x.size(dim=0)):
            m = self.NN_model(x[i].detach()).float() # normed x |-> NN normed y 
            m = self.y_norm_transform.untransform(m) # NN normed y -> real y
            m = self.outcome_transform(m)[0] # real y -> standardized y
            out.append(m.squeeze())
            
        self.outcome_transform.train()
        return torch.stack(out,dim=0)
    
# surrogate model as prior 
class Surrogate(Mean):
    def __init__(self, input_transform, outcome_transform):
        super(Surrogate, self).__init__()
        self.input_transform = input_transform 
        self.outcome_transform = outcome_transform

    def forward(self, x):
        """
        takes in transformed x, returns transformed y (retaining gradient) 
        """
        self.input_transform.eval()
        self.outcome_transform.eval()
        
        x = self.input_transform.untransform(x) # normed x -> real x 
        out = []
        
        if (x.dim() == 2):
            x = x.unsqueeze(dim=0) # shape x into (batch_size = 1, n_samples, n_var) if necessary 
        for i in range(x.size(dim=0)):
            m = evaluate(x[i].detach()).float() # real x |-> real y
            m = self.outcome_transform(m)[0] # real y -> standardized y
            out.append(m.squeeze())
            
        self.input_transform.train() 
        self.outcome_transform.train()
        return torch.stack(out,dim=0)

### BO trials

In [ ]:
n_trials = 5
n_steps = 30

BO_runs = ['constant (default)','ground truth', 'model3_1hidden_20nodes_500epoch_0.02', 'model4_3hidden_30nodes_500epoch_0.01',
          'model5_4hidden_40nodes_1000epoch_0.01_-5'] # names of runs 

transformer_x = botorch.models.transforms.input.Normalize(n_var, bounds = bounds.transpose(0,1))
NN_y_transformers = ['transformer_y_3^9_norm', 'transformer_y_4^9_norm', 'transformer_y_5^9_-5_norm']

filename = 'surr_const_model5_-5_2.0_2'

best_y_lists = [] # best y value over iterations of each run
x_configs = [] # changes in parameter values over iterations of each run 
for i in range(len(BO_runs)):
    best_y_lists.append([])
    x_configs.append([])
             
run_BO = [True, True, False, False, True] # whether or not to run the type of BO

for i in range(n_trials):
    #for each trial, new training samples are generated, and each BO run type is executed
    
    train_x = torch.zeros((n_samples, n_var)) 
    for j in range(n_var):
        train_x[:,j] = torch.tensor(np.random.uniform(bounds[j,0],bounds[j,1],(n_samples,)))

    train_y = evaluate(train_x).detach().reshape(-1,1)
    # print(train_x, train_y)
    
    transformer_y_list = [] # outcome_transform modules for each type of run
    for j in range(len(BO_runs)):
        transformer_y_list.append(botorch.models.transforms.outcome.Standardize(1))

    priors = [None, Surrogate(transformer_x, transformer_y_list[1]),
             CustomMean(BO_runs[2], NN3_prior(), NN_y_transformers[0], transformer_y_list[2]),
             CustomMean(BO_runs[3], NN4_prior(), NN_y_transformers[1], transformer_y_list[3]),
             CustomMean(BO_runs[4], NN5_prior(), NN_y_transformers[2], transformer_y_list[4])] # the priors used for each type of run 
        
    print(f'iter {i}')
    for run_index in range (len(BO_runs)):

        if run_BO[run_index] == True:
            print(f'running {BO_runs[run_index]}')
            prior_best_y, prior_x_config = BayesianOptimization(train_x, 
                                                         train_y, 
                                                         n_steps=n_steps, 
                                                         prior=priors[run_index], 
                                                         transformer_x=transformer_x,
                                                         transformer_y=transformer_y_list[run_index])
            best_y_lists[run_index].append(prior_best_y)
            x_configs[run_index] = prior_x_config
        torch.save({'BO_runs': BO_runs, 'run_BO': run_BO, 'x_configs': x_configs, 'best_y_lists': best_y_lists}, './results/' + filename + '.pt')

iter 0
running constant (default)
running ground truth
running model5_4hidden_40nodes_1000epoch_0.01_-5
iter 1
running constant (default)
running ground truth


/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  NumericalWarning,
/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  NumericalWarning,
/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  NumericalWarning,
/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negativ

running model5_4hidden_40nodes_1000epoch_0.01_-5
iter 2
running constant (default)
running ground truth
running model5_4hidden_40nodes_1000epoch_0.01_-5


/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/botorch/fit.py:148: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(w.message, w.category)
/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/botorch/fit.py:148: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(w.message, w.category)


iter 3
running constant (default)
running ground truth


/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  NumericalWarning,
/sdf/home/x/xu123456/miniconda3/envs/baxenv/lib/python3.7/site-packages/gpytorch/distributions/multivariate_normal.py:263: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  NumericalWarning,


running model5_4hidden_40nodes_1000epoch_0.01_-5


In [ ]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""